In [1]:
from time import process_time_ns
import plotly.graph_objects as go
import copy_ as fnc
from functions_straddle import sharpie_trades
from process import fetch_options_
dt = 'date_timestamp'
import psycopg2;import pandas as pd;import process as dp;import functions_straddle as fn;from pandas.errors import SettingWithCopyWarning;import GeeksCalculator as gk;import importlib;import warnings;importlib.reload(dp);importlib.reload(fn);importlib.reload(gk);warnings.simplefilter(action='ignore', category=FutureWarning);warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)
conn = psycopg2.connect(dbname="qdap_test",user="amt",password="your_password",host="192.168.2.23",port="5432")
cursor = conn.cursor()

In [10]:
importlib.reload(dp); importlib.reload(fn)
all_trades, all_execs , trade_id = {} , {} ,0

ranges = pd.date_range(start='2024-01-01', end='2024-05-31')
st = '2020-01-29'
# ranges = pd.date_range(start=st, end=st)

eh = 9
em = 30

tradeid = 0

all_running_pnl, all_call_movement, all_put_movement, all_call_iv, all_put_iv = [],[],[],[],[]
list_symbol = ['BANKNIFTY']
for symbol in list_symbol:
    all_trades[symbol],all_execs[symbol] = [] , []
    for d_t in ranges:
        # try:
            futures = dp.FetchFuturesByDate(cursor,symbol,d_t)
            if not futures.empty:
                options = dp.FetchOptionsByDateCsv(symbol,d_t.date())
                print(symbol,str(d_t.date()))
                gen_call, gen_put = options[options['opt_type'] == 'CE'], options[options['opt_type'] == 'PE']
                call, put = dp.fetch_call_put_(options)
                call.ffill(inplace=True) ; put.ffill(inplace=True)
                tradeid,execs, trades, running_pnl, net_pnl, capital, rollover, positions, difference, call_movement, put_movement,delta_put, delta_call, call_iv, put_iv, iv = fn.generate_trades_syn(eh,em,tradeid,futures, call, put, gen_call, gen_put, io=0,DeltaLB=0.2,delta_range=0.1,DeltaUB=0.55, buying='close', selling='close', initial_capital=100000)
                all_running_pnl.extend(running_pnl);    all_call_movement.extend(call_movement);  all_put_movement.append(put_movement);   all_call_iv.extend(call_iv);  all_put_iv.extend(put_iv);    all_trades[symbol].extend(trades);  all_execs[symbol].extend(execs)
                streams,labels = [running_pnl,iv,call_movement,put_movement] , ['running_pnl','iv','cs','ps'] ; dp.plotter(streams,labels,filename=f'Graph_{d_t.date()}.html')
                if len(trades) == 0:
                    print(f"No trades took place on {d_t.date()}")
        # except Exception as e:
        #     # Print the error message
        #     print(f"Error on date {d_t} for symbol {symbol}: {str(e)}")
        #     # Print the traceback for more details


BANKNIFTY 2024-01-01
BANKNIFTY 2024-01-02
BANKNIFTY 2024-01-03
No trades took place on 2024-01-03
BANKNIFTY 2024-01-04
BANKNIFTY 2024-01-05
BANKNIFTY 2024-01-08
BANKNIFTY 2024-01-09
BANKNIFTY 2024-01-10
No trades took place on 2024-01-10
BANKNIFTY 2024-01-11
BANKNIFTY 2024-01-12
BANKNIFTY 2024-01-15
BANKNIFTY 2024-01-16
BANKNIFTY 2024-01-17
No trades took place on 2024-01-17
BANKNIFTY 2024-01-18
BANKNIFTY 2024-01-19
BANKNIFTY 2024-01-20
BANKNIFTY 2024-01-23
BANKNIFTY 2024-01-24
BANKNIFTY 2024-01-25
No trades took place on 2024-01-25
BANKNIFTY 2024-01-29
BANKNIFTY 2024-01-30
BANKNIFTY 2024-01-31
No trades took place on 2024-01-31
BANKNIFTY 2024-02-01
BANKNIFTY 2024-02-02
BANKNIFTY 2024-02-05
BANKNIFTY 2024-02-06
BANKNIFTY 2024-02-07
No trades took place on 2024-02-07
BANKNIFTY 2024-02-08
BANKNIFTY 2024-02-09
BANKNIFTY 2024-02-12
BANKNIFTY 2024-02-13
BANKNIFTY 2024-02-14
No trades took place on 2024-02-14
BANKNIFTY 2024-02-15
BANKNIFTY 2024-02-16
BANKNIFTY 2024-02-19
BANKNIFTY 2024-02-20

In [ ]:
pri_ = {'id':1,'time': 1,'opt_type':1,'log':1,'delta':1,'strike': 1, 'price': 1,'pnl': 0, 'trade_type': 1,'transaction_cost': 0, 'slippage': 0,}
fn.print_execs_to_html(all_execs['BANKNIFTY'],pri_,filename=f'all exec log.html')

columns_to_print = {'entry_time':1 , 'entry_price':1 ,'exit_price':1, 'exit_time' : 1 ,  'time':1 , 'movement': 0, 'type':1 , 'pnl':1,'strike':1,'entry_delta':1,'exit_delta':1,'sell_id':1,'buy_id':1}
fn.print_trades_to_html(all_trades['BANKNIFTY'], columns_to_print , filename=f'all trade logs.html')

sharpie, stats = fn.sharpie_straddle(all_trades['BANKNIFTY'])
print(sharpie)


In [10]:
sample  =  dp.FetchOptionsByDate(cursor,'NIFTY','2020-01-01')

In [11]:
sample.head()

,symbol,date_timestamp,open,high,low,close,volume,opt_type,strike,expiry_type,id,expiry
0,NIFTY,2020-01-01 09:15:00,144,151,141,142,4575,PE,12350,IW1,348354198,2020-01-02
1,NIFTY,2020-01-01 09:15:00,108,118,107,115,35775,CE,12100,IW1,348350073,2020-01-02
2,NIFTY,2020-01-01 09:15:00,105,108,96,98,44400,PE,12300,IW1,348353448,2020-01-02
3,NIFTY,2020-01-01 09:15:00,0,0,0,0,1650,PE,11600,IW1,348347278,2020-01-02
4,NIFTY,2020-01-01 09:15:00,0,0,0,0,3450,CE,12700,IW1,348356286,2020-01-02
